## Monte Carlo swaption pricer

Pricing swaptions using Monte-Carlo

-Able to select between random number generator/Sobol

-Results also with annuity factor as control variate instrument

In [1]:
import os
import sys

In [2]:
current_dir = os.getcwd()
qg_dir = os.path.join(current_dir, "..")

sys.path.append(qg_dir)

In [3]:
from quassigaussian.montecarlo.simulations import ProcessSimulatorTerminalMeasure
from quassigaussian.montecarlo.monte_carlo_pricer import monte_carlo_pricer_terminal_measure
from quassigaussian.parameters.volatility.local_volatility import LinearLocalVolatility
from quassigaussian.products.instruments import Swap, Swaption
from quassigaussian.montecarlo.control_variate import apply_control_variate_annuity
from quassigaussian.products.pricer import SwapPricer, SwaptionPricer, find_implied_black_vola
import pandas as pd
import os
import numpy as np
from qgtests.utis import get_mock_yield_curve_const
from report.directories import output_data_raw, date_timestamp
from report.utils import get_nonexistant_path


In [4]:
number_paths = 2**14
number_time_steps = 2**10

random_number_generator_type = "sobol"
random_number_generator_type = "normal"


In [5]:
curve_rate = 0.06
kappa = 0.03
swaption_expiry=5
swap_maturity = 10

initial_curve = get_mock_yield_curve_const(rate=curve_rate)


loca_vola = LinearLocalVolatility.from_const(swap_maturity, 0.4, 0.01, 0.1)

swap_pricer = SwapPricer(initial_curve, kappa)
swap = Swap(swaption_expiry, swap_maturity, 0.5)
swaption_pricer = SwaptionPricer(swap_pricer)


In [ ]:
# Simulating XY processes

bond_measure = swap.bond_T0

process_simulator = ProcessSimulatorTerminalMeasure(number_paths, number_time_steps,
                                             swaption_expiry / number_time_steps,
                                             random_number_generator_type, bond_measure,
                                                    swap_pricer.bond_pricer, nr_processes=6,
                                                    n_scrambles=64)

# able to do multiprocessing in Python
result_obj = process_simulator.simulate_xy(kappa, loca_vola, parallel_simulation=True)


In [ ]:
# Defining coupon grid for swaptions
coupon_grid = [0, +0.0025, -0.0025, +0.005, -0.005, +0.01, -0.01, 0.015, -0.015, 0.02, -0.02, 0.025, -0.025]

atm_swap_price = swap_pricer.price(swap, 0, 0, 0)
strike_grid = [atm_swap_price+coupon for coupon in coupon_grid]


In [ ]:
## Analyzing resuts

def calculate_implied_vola():    
    output_all_df_ls = []
    for strike in strike_grid:
        swaption = Swaption(swaption_expiry, strike, swap)

        swaption_value_paths = monte_carlo_pricer_terminal_measure(result_obj, swaption, swaption_pricer)
        last_mc_time = result_obj.time_grid[-1]
        
        # Use Annuity as control variate instrument
        swaption_value_paths_cv2 = apply_control_variate_annuity(last_mc_time, result_obj.x[:, -1],
                                                        result_obj.y[:, -1], swaption_value_paths,
                                                        swap.annuity, swap_pricer.annuity_pricer,
                                                         swap_pricer.annuity_pricer.bond_pricer.initial_curve)

        swaption_value_mean = swaption_value_paths.mean()
        std, swaption_value_error = result_obj.calculate_std_error(swaption_value_paths, result_obj.n_scrambles)

        # swaption_value_mean_cv = swaption_value_paths_cv.mean()
        # std, swaption_value_error_cv = result_obj.calculate_std_error(swaption_value_paths_cv, result_obj.n_scrambles)

        swaption_value_mean_cv = swaption_value_paths_cv2.mean()
        std, swaption_value_error_cv = result_obj.calculate_std_error(swaption_value_paths_cv2,
                                                                      result_obj.n_scrambles)

        bond_pricer = swap_pricer.bond_pricer
        output_data = {"number_paths": number_paths, "number_time_steps": number_time_steps,
                       "random_number_generator_type": random_number_generator_type, "expiry": swaption_expiry,
                       "maturity": swap_maturity, "strike": strike, "atm strike": atm_swap_price,
                       "moneyness": strike-atm_swap_price,
                       "curve_rate": curve_rate, "kappa": kappa, "swaption value": swaption_value_mean,
                       "swaption value error": swaption_value_error,
                       "swaption value cv": swaption_value_mean_cv,
                       "swaption value error cv": swaption_value_error_cv,

                       "implied_vola": find_implied_black_vola(swaption_value_mean, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_max": find_implied_black_vola(swaption_value_mean+swaption_value_error,
                                                                   swaption, swap_pricer, bond_pricer),
                       "implied_vola_min": find_implied_black_vola(swaption_value_mean-swaption_value_error,
                                                                   swaption, swap_pricer, bond_pricer),
                       "implied_vola_cv": find_implied_black_vola(swaption_value_mean_cv, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_cv_max": find_implied_black_vola(swaption_value_mean_cv + swaption_value_error_cv, swaption,
                                                               swap_pricer, bond_pricer),
                       "implied_vola_cv_min": find_implied_black_vola(swaption_value_mean_cv - swaption_value_error_cv, swaption,
                                                               swap_pricer, bond_pricer)}

        output_df_new = pd.DataFrame(output_data, index=[0])
        output_all_df_ls.append(output_df_new)

    return output_all_df_ls


In [ ]:
output_all_df_ls = calculate_implied_vola()
output_all_df = pd.concat(output_all_df_ls)
output_all_df.sort_values("moneyness")